In [1]:
import pandas as pd
import numpy as np
import copy
import random

# Camiones y Camión

In [2]:
df_clientes = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="clientes")
df_pedidos = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [3]:
df_camiones = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="camiones")
df_camiones

,camion,carga_max,clientes_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


In [4]:
df_distancias = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="distancias", index_col=[0])
df_distancias

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
Cliente,,,,,,,,,,,,,,,
A,0.000000,1.300481,0.942292,0.831757,1.190134,0.820391,2.138542,2.178008,1.489282,2.169046,2.899109,2.047357,1.491853,2.250309,1.767958
B,1.300481,0.000000,0.968635,0.511577,1.701683,0.835731,2.086012,1.486356,1.244076,1.120697,2.068003,0.747466,0.693247,1.833682,2.088735
C,0.942292,0.968635,0.000000,0.567701,1.965609,1.212640,2.710010,2.369018,1.900745,2.085834,3.008383,1.563518,1.554526,2.621602,2.501215
D,0.831757,0.511577,0.567701,0.000000,1.537098,0.698118,2.172654,1.805154,1.346806,1.569917,2.459347,1.237830,0.994192,2.055756,2.026486
E,1.190134,1.701683,1.965609,1.537098,0.000000,0.876150,1.049474,1.565477,0.826470,1.917289,2.253598,2.315795,1.334625,1.383229,0.579504
F,0.820391,0.835731,1.212640,0.698118,0.876150,0.000000,1.498406,1.359521,0.719408,1.389120,2.078832,1.510223,0.704151,1.478470,1.330847
G,2.138542,2.086012,2.710010,2.172654,1.049474,1.498406,0.000000,1.049438,0.845302,1.667396,1.510842,2.429644,1.439609,0.594991,0.613642
H,2.178008,1.486356,2.369018,1.805154,1.565477,1.359521,1.049438,0.000000,0.765932,0.658239,0.727953,1.546591,0.816165,0.499332,1.482584
I,1.489282,1.244076,1.900745,1.346806,0.826470,0.719408,0.845302,0.765932,0.000000,1.106023,1.485664,1.667471,0.639137,0.764083,0.934229


In [ ]:
for cod in list(df_clientes.index)[1:]:
    print(round(((df_clientes.loc["A"]["Coord x"] - df_clientes.loc[cod]["Coord x"])**2 + (df_clientes.loc["A"]["Coord y"] - df_clientes.loc[cod]["Coord y"])**2)**(1/2), 1))

In [151]:
class Camion(object):
    
    def __init__(self, ix, carga_max, pedidos_max, dist_max):
        self.ix = ix
        self.carga_max = carga_max
        self.pedidos_max = pedidos_max
        self.dist_max = dist_max
        self.pedidos_asignados = {}
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
    def __str__(self):
        return f'Camión {self.ix}\nCarga Total {self.carga_total} tn\nPedidos {[ped.ix for ped in self.pedidos_asignados.values()]}\nCosto Total {self.get_costo()}\nCosto por tn {self.get_costo_tn()}'
    
    def __repr__(self):
        return f'Camión {self.ix}\nCarga Total {self.carga_total} tn\nPedidos {[ped.ix for ped in self.pedidos_asignados.values()]}\nCosto Total {self.get_costo()}\nCosto por tn {self.get_costo_tn()}'
    
    def _check_pedido_carga(self, pedido):
        return (self.carga_total + pedido.carga) <= self.carga_max
    
    def _check_pedido_cantidad(self, pedido):
        return self.cantidad_pedidos < self.pedidos_max
    
    def _check_pedido_distancia(self, pedido):
        dist_check = [pedido.distancia(other) <= self.dist_max for other in self.pedidos_asignados.values()]
        return all(dist_check)
    
    def check_nuevo_pedido(self, pedido):
        #return not pedido.asignado and self._check_pedido_carga(pedido) and self._check_pedido_cantidad(pedido) and self._check_pedido_distancia(pedido)
        return self._check_pedido_carga(pedido) and self._check_pedido_cantidad(pedido) and self._check_pedido_distancia(pedido)
        
    def check_intercambio_pedido(self, pedido):
        ix_pedidos_intercambio = []
        
        for pedido_original in self.pedidos_asignados.values():
            
            nueva_carga = self.carga_total + pedido.carga - pedido_original.carga
            distancias = [pedido.distancia(pedido_restante) <= self.dist_max for pedido_restante in self.pedidos_asignados.values() if pedido_restante.ix != pedido_original.ix]
            
            if nueva_carga <= self.carga_max and all(distancias):
                
                ix_pedidos_intercambio.append(pedido_original.ix)
                
        return ix_pedidos_intercambio
                
        
        
    def add_pedido(self, pedido):
        if not pedido.asignado:
            self.pedidos_asignados[pedido.ix] = pedido
            self.carga_total += pedido.carga
            self.cantidad_pedidos += 1
            pedido.asignado = True
            pedido.camion_ix = self.ix
        #     return True
        # else:
        #     return False
            
    def add_pedido_checked(self, pedido):
        if self.check_nuevo_pedido(pedido):
            self.add_pedido(pedido)
        
    
    def remove_pedido(self, pedido_ix):
        self.pedidos_asignados.get(pedido_ix).asignado = False
        self.pedidos_asignados.get(pedido_ix).camion_ix = None
        self.carga_total -= self.pedidos_asignados.get(pedido_ix).carga
        self.cantidad_pedidos -= 1
        self.pedidos_asignados.pop(pedido_ix)
    
    def get_carga_total(self):
        return self.carga_total
    
    def get_costo(self):
         
        if self.carga_total == 0:
            costo = 5000
        elif self.carga_total <= 4:
            costo = 5600
        elif self.carga_total > 4 and self.carga_total < 6.5:
            costo = 1400*self.carga_total
        elif self.carga_total >= 6.5 and self.carga_total < 9.5:
            costo = 1200*self.carga_total
        else:
            costo = 1000*self.carga_total
                
        return costo 
    
    def get_costo_tn(self):
        if self.carga_total != 0:
            return self.get_costo()/self.carga_total
        else:
            return None
    
    def count_pedidos(self):
        return self.cantidad_pedidos
    
    def get_ix_pedidos(self):
        return list(self.pedidos_asignados.keys())
    
    def reset_pedidos(self):
        self.pedidos_asignados.clear()
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
        
        
        
class Pedido(object):
    
    def __init__(self, ix, x, y, carga):
        self.ix = ix
        self.x = x
        self.y = y
        self.carga = carga
        self.asignado = False
        self.camion_ix = None
        
    def __str__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}\nAsignado a Camion {self.camion_ix}'
    
    def __repr__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}\nAsignado a Camion {self.camion_ix}'
    
    def get_ix(self):
        return self.ix
    
    def get_carga(self):
        return self.carga
    
    def distancia(self, other):
        dist = ((self.x - other.x)**2 + (self.y - other.y)**2)**(1/2)
        return round(dist, 1)
    
    
    
class Ruteo(object):
    
    def __init__(self, camiones_dict, pedidos_dict, random_state):
        self.camiones = camiones_dict
        self.pedidos = pedidos_dict
        self.random_state = random_state
        
    def get_ix_camiones(self):
        return list(self.camiones.keys())
    
    def get_ix_pedidos(self):
        return list(self.pedidos.keys())
    
    def get_camiones(self):
        return list(self.camiones.values())
    
    def get_pedidos(self):
        return list(self.pedidos.values())
    
    def get_camion(self, ix):
        return self.camiones.get(ix)
    
    def get_pedido(self, ix):
        return self.pedidos.get(ix)
    
        
    def get_solucion_inicial(self, mode):
        
        random.seed(self.random_state)
        #np.random.seed(self.random_state)
        
        if mode == "1":
            for camion in self.get_camiones():
                for pedido in self.get_pedidos():
                    camion.add_pedido_checked(pedido)
                    
        elif mode == "2":     
            # CREO UNA LISTA DE IX DE PEDIDOS MEZCLADA AL AZAR 
            # PARA CADA PEDIDO INTENTO METERLO EN CADA CAMION AL AZAR (CAMIONES MEZCLADOS)
            ix_pedidos = random.sample(self.get_ix_pedidos(), len(self.get_ix_pedidos()))
            
            for ix_pedido in ix_pedidos:
                ix_camiones = random.sample(self.get_ix_camiones(), len(self.get_ix_camiones()))
                for ix_camion in ix_camiones:
                    self.get_camion(ix_camion).add_pedido_checked(pedido)
            
        else:
            for camion in self.get_camiones():
                for pedido in self.get_pedidos():
                    camion.add_pedido_checked(pedido)
                    
                    
    def generar_vecino(self, random_state):
        
        # Genera una lista de repeticiones (deep copies) de la solucion inicial.
        # Usa una estrategia para modificar cada una de las copias para tener soluciones vecinas factibles.
        
        #np.random.seed(random_state)
        random.seed(random_state)
        
        camion_index = random.randint(0, len(self.camiones)-1)
        pedido_index = random.randint(0, len(self.pedidos)-1)
        
        pass
        
    
    # PARA CAMBIAR TODA ESTA PARTE
    # GENERAR UN DATAFRAME CON TODOS LOS DATOS.
    def set_results(self):
        self.set_carga_total()
        self.set_costo_camiones()
        self.set_costo_oportunidad()
        self.set_costo_total()
        self.set_costo_total_tn()
        self.set_ahorro()
        
    def get_results(self):
        self.set_results()
        print("--Resultados--")
        print(f"Carga Total {self.carga_total} tn")
        print(f"Costo Camiones {self.costo_camiones} $")
        print(f"Costo Oportunidad {self.costo_oport} $")
        print(f"Costo Total {self.costo_total} $")
        print(f"Costo Total por tn {self.costo_total_tn} $")
        print(f"Ahorro {self.ahorro*100}%")
    
    def set_carga_total(self):
        self.carga_total = sum([camion.get_carga_total() for camion in self.camiones.values()])
        
    def set_costo_camiones(self):
        self.costo_camiones = sum([camion.get_costo() for camion in self.camiones.values()])
        
    def set_costo_oportunidad(self):
        carga_no_asignada = sum([pedido.get_carga() for pedido in self.pedidos.values() if not pedido.asignado])
        self.costo_oport = carga_no_asignada*3000
        
    def set_costo_total(self):
        self.costo_total = self.costo_camiones + self.costo_oport
    
    def set_costo_total_tn(self):
        self.costo_total_tn = round(self.costo_total/self.carga_total, 2)
        
    def set_ahorro(self):
        self.ahorro = round((self.costo_total_tn - 1200)/1200, 4)
        
        
def generar_vecinos(sol_inicial, n, random_state):
    # Genera una lista de repeticiones (deep copies) de la solucion inicial.
    # Usa una estrategia para modificar cada una de las copias para tener soluciones vecinas factibles.
    random.seed(random_state)
    #np.random.seed(random_state)
    vecinos = []
    
    for i in range(n):
        vecino = copy.deepcopy(sol_inicial)
        
        camion_index = random.randint(0, len(vecino.camiones)-1)
        pedido_index = random.randint(0, len(vecino.pedidos)-1)
        


        

In [63]:
random.sample(ruteo2.get_ix_pedidos(), len(ruteo2.get_ix_pedidos()))

12

In [12]:
x = {"A":2, "B":3}

x.get("A")

2

In [155]:
ruteo2 = copy.deepcopy(ruteo)

In [156]:
random.seed(42)

In [168]:
# Elijo un índice al azar de la lista de pedidos.
pedido1_index = random.choice(ruteo2.get_ix_pedidos())
print(pedido1_index)

# Extraigo dicho pedido.
pedido1 = ruteo2.get_pedido(pedido1_index)
# Obtengo los índices de camiones en los que entraría este pedido.
ix_camion_check = [camion.ix for camion in ruteo2.get_camiones() if camion.check_nuevo_pedido(pedido1)]
#i_peds_similares = [pedido.ix for pedido in ruteo2.get_pedidos() if pedido.get_carga() >= pedido1.get_carga()]

print(pedido1)
print(ix_camion_check)
print(len(ix_camion_check))
#print(i_peds_similares)
#print(len(i_peds_similares))
print(ruteo2.get_results())

# SI YA ESTA ASIGNADO:
if pedido1.asignado:
    
    # ELIMINAMOS AL PEDIDO A REASIGNAR DE SU CAMION.
    ix_camion_pedido1 = pedido1.camion_ix
    ruteo2.get_camion(ix_camion_pedido1).remove_pedido(pedido1.ix)
    
    
    # SIEMPRE VOY A INTENTAR COMPLETAR CAMIONES
    # SI ENTRA EN OTRO CAMION, CON CIERTA PROBABILIDAD SE ASIGNA A DICHO CAMION.
    #if len(i_camion_check) > 0 and random.uniform(0,1) > 0.5:
    if len(ix_camion_check) > 0:
        ix_camion = random.choice(ix_camion_check)
        # NO HACE FALTA CHEQUEAR.
        ruteo2.get_camion(ix_camion).add_pedido(pedido1)
    
    # SI NO ENTRA DIRECTAMENTE EN OTRO CAMION REEMPLAZAR UN PEDIDO DE OTRO CAMION:
    # NO DEBERIA REEMPLAZAR A UN PEDIDO QUE NO ESTE ASIGNADO.
    else:
        pedidos_reemplazables = []
        for camion in ruteo2.get_camiones():
            pedidos_reemplazables += camion.check_intercambio_pedido(pedido1)
        
        pedidos_posibles = []
        for otro_pedido in pedidos_reemplazables:
            if ruteo2.get_camion(ix_camion_pedido1).check_nuevo_pedido(otro_pedido):
                pedidos_posibles += otro_pedido
        
        pedido2 = ruteo2.get_pedido(random.choice(pedidos_posibles))               
        ix_camion_pedido2 = pedido2.camion_ix
        ruteo2.get_camion(ix_camion_pedido2).remove_pedido(pedido2.ix)
        
        ruteo2.get_camion(ix_camion_pedido2).add_pedido(pedido1)
        ruteo2.get_camion(ix_camion_pedido1).add_pedido(pedido2)

# SI EL PEDIDO NO ESTA ASIGNADO:
# SIEMPRE CONVIENE INTERNTAR METERLO.
else:
    # PRUEBA SI ENTRA EN ALGUN CAMION SIN CAMBIAR.
    if len(ix_camion_check) > 0:
        ix_camion = random.choice(ix_camion_check)
        ruteo2.get_camion(ix_camion).add_pedido(pedido1)
        
    # SI NO ENTRA DE MANERA DIRECTA EN OTRO CAMION TIENE QUE CAMBIAR CON OTRO PEDIDO Y DEJARLO SIN ASIGNAR.
    else:
        pedidos_reemplazables = []
        for camion in ruteo2.get_camiones():
            pedidos_reemplazables += camion.check_intercambio_pedido(pedido1)
            
        pedido2 = ruteo2.get_pedido(random.choice(pedidos_reemplazables))               
        ix_camion_pedido2 = pedido2.camion_ix
        ruteo2.get_camion(ix_camion_pedido2).remove_pedido(pedido2.ix)
        
        ruteo2.get_camion(ix_camion_pedido2).add_pedido(pedido1)
        
ruteo2.get_results()

D
Pedido D
Carga 5 tn
Asignado True
Asignado a Camion 1
[5]
1
--Resultados--
Carga Total 55 tn
Costo Camiones 59400 $
Costo Oportunidad 0 $
Costo Total 59400 $
Costo Total por tn 1080.0 $
Ahorro -10.0%
None
--Resultados--
Carga Total 55 tn
Costo Camiones 60400 $
Costo Oportunidad 0 $
Costo Total 60400 $
Costo Total por tn 1098.18 $
Ahorro -8.48%


In [96]:
x = []
x += ["A", "B"]
x += ["C", "D"]
x

['A', 'B', 'C', 'D']

In [30]:
ruteo.camiones[0].get_ix_pedidos()

['A', 'B', 'E']

In [7]:
[row.camion for _, row in df_camiones.iterrows()]

[1, 2, 3, 4, 5, 6]

In [152]:
pedidos_tomados = "pedidos3"

#lista_camiones = [Camion(ix=row.camion, carga_max=row.carga_max, pedidos_max=row.clientes_max, dist_max=row.dist_max) for _, row in df_camiones.iterrows()]
#lista_pedidos = [Pedido(ix=row.cliente, x=row.coord_x, y=row.coord_y, carga=row[pedidos_tomados]) for _, row in df_pedidos.iterrows() if row[pedidos_tomados] != 0]

dict_camiones = {row.camion:Camion(ix=row.camion, carga_max=row.carga_max, pedidos_max=row.clientes_max, dist_max=row.dist_max) for _, row in df_camiones.iterrows()}
dict_pedidos = {row.cliente:Pedido(ix=row.cliente, x=row.coord_x, y=row.coord_y, carga=row[pedidos_tomados]) for _, row in df_pedidos.iterrows() if row[pedidos_tomados] != 0}

ruteo = Ruteo(dict_camiones, dict_pedidos, 42)

In [153]:
ruteo.get_solucion_inicial(mode="1")

In [154]:
for camion in ruteo.camiones.values():
    print(camion)
    print("------------")
    
for pedido in ruteo.pedidos.values():
    print(pedido)
    print("------------")
    
print("-------------")
ruteo.get_results()

Camión 1
Carga Total 12 tn
Pedidos ['A', 'B', 'E']
Costo Total 12000
Costo por tn 1000.0
------------
Camión 2
Carga Total 11 tn
Pedidos ['C', 'D']
Costo Total 11000
Costo por tn 1000.0
------------
Camión 3
Carga Total 12 tn
Pedidos ['F', 'G', 'I']
Costo Total 12000
Costo por tn 1000.0
------------
Camión 4
Carga Total 11 tn
Pedidos ['H', 'J']
Costo Total 11000
Costo por tn 1000.0
------------
Camión 5
Carga Total 9 tn
Pedidos ['K', 'L']
Costo Total 10800
Costo por tn 1200.0
------------
Camión 6
Carga Total 0 tn
Pedidos []
Costo Total 5000
Costo por tn None
------------
Pedido A
Carga 5 tn
Asignado True
Asignado a Camion 1
------------
Pedido B
Carga 5 tn
Asignado True
Asignado a Camion 1
------------
Pedido C
Carga 6 tn
Asignado True
Asignado a Camion 2
------------
Pedido D
Carga 5 tn
Asignado True
Asignado a Camion 2
------------
Pedido E
Carga 2 tn
Asignado True
Asignado a Camion 1
------------
Pedido F
Carga 6 tn
Asignado True
Asignado a Camion 3
------------
Pedido G
Carga 3 tn

In [84]:
ruteo2 = copy.deepcopy(ruteo)
print(ruteo2.get_camion(1))
print(ruteo2.get_pedido("A"))
print("------")
ruteo2.get_camion(1).remove_pedido("A")
print(ruteo2.get_camion(1))
print(ruteo2.get_pedido("A"))
print("------")
print(ruteo.get_camion(1))
print(ruteo.get_pedido("A"))

Camión 1
Carga Total 12 tn
Pedidos ['A', 'B', 'E']
Costo Total 12000
Costo por tn 1000.0
Pedido A
Carga 5 tn
Asignado True
Asignado a Camion 1
------
Camión 1
Carga Total 7 tn
Pedidos ['B', 'E']
Costo Total 8400
Costo por tn 1200.0
Pedido A
Carga 5 tn
Asignado False
Asignado a Camion None
------
Camión 1
Carga Total 12 tn
Pedidos ['A', 'B', 'E']
Costo Total 12000
Costo por tn 1000.0
Pedido A
Carga 5 tn
Asignado True
Asignado a Camion 1


In [90]:
ruteo.get_camion(1).check_intercambio_pedido(ruteo.get_pedido("H"))

['A']

In [246]:
pedido = Pedido(ix="C", x=2, y=2, carga=11)

[pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()]

[False, False]

In [247]:
any([pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()])

False

In [250]:
not False and False

False

In [221]:
camion1.pedidos_asignados

{}

In [53]:
x = {}
x["A"] = Pedido("A")
x["B"] = Pedido("B")
print(x)
x.pop("B")
print(x)
x.clear()
print(x)

{'A': Pedido A, 'B': Pedido B}
{'A': Pedido A}
{}


In [238]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)
camion2 = Camion(ix=2, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido_checked(Pedido(ix="A", x=1, y=1, carga=4))
camion1.add_pedido_checked(Pedido(ix="B", x=3, y=3, carga=5))
camion2.add_pedido_checked(Pedido(ix="C", x=1, y=1, carga=11))

print(camion1.count_pedidos())
print(camion2.count_pedidos())
print(camion1.get_carga_total())
print(camion2.get_carga_total())
print(camion1.get_costo())
print(camion2.get_costo())
print(camion1.get_costo_tn())
print(camion2.get_costo_tn())


1
1
5
11
7000
11000
1400.0
1000.0


In [263]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido_checked(Pedido(ix="A", x=1, y=1, carga=4))
print(camion1.get_carga_total())
camion1.add_pedido_checked(Pedido(ix="B", x=1, y=1, carga=5))
print(camion1.get_carga_total())
camion1.add_pedido_checked(Pedido(ix="C", x=1, y=1, carga=4))
print(camion1.get_carga_total())

camion1.pedidos_asignados

True
True
True
True
True
4
True
True
True
True
True
9
True
False
True
True
False
9


{'A': Pedido A, 'B': Pedido B}

In [261]:
x = 1
x -= 1
print(x)

0


## Pruebas con datos reales

In [279]:
camion_prueba = df_camiones.iloc[0,:]
camion_prueba

camion           1
carga_max       12
clientes_max     3
dist_max         2
Name: 0, dtype: int64

In [280]:
camion = Camion(ix=camion_prueba.camion,
                carga_max=camion_prueba.carga_max,
                pedidos_max=camion_prueba.clientes_max,
                dist_max=camion_prueba.dist_max)

In [281]:
pedido_prueba = df_pedidos.iloc[0,0:2]
pedido_prueba

cliente_prueba = df_clientes.iloc[0]
cliente_prueba

Coord x    0.257571
Coord y    1.803726
Name: A, dtype: float64

In [282]:
pedido = Pedido(ix=pedido_prueba.Cliente,
                x=cliente_prueba["Coord x"],
                y=cliente_prueba["Coord y"],
                carga=pedido_prueba["Pedidos #1"])

In [283]:
camion.add_pedido_checked(pedido)

True
True
True
True
True


Para controlar cómo copiar y guardar elementos en los camiones. Pensar qué pasaría si cargo muchos objetos en memoria.

In [284]:
# Pruebas de memoria para realizar cambios -> Ojo con modificar objetos.
camion1 = Camion("1", 2, 2, 2)
#camion2 = copy.copy(camion1)
camion2 = copy.deepcopy(camion1)
pedido1 = Pedido("A", 1, 1, 4)

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

#pedido.carga = 5
#camion.pedidos_asignados.get("A").carga = 5
camion2.pedidos_asignados.get("A").carga = 5

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

AttributeError: 'NoneType' object has no attribute 'carga'

In [175]:
class A(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class B(A):
    
    def __init__(self):
        self.x = A.x

In [176]:
a = A(1,2)
b = B(a)

TypeError: __init__() takes 1 positional argument but 2 were given